# Project 2 - Machine Learning, Fall 2022

Deadline: 11th of December 2022, 23:59

To do this project you have to complete this Jupyter notebook and send it via Discord.

The total number of points allocated for this project is 10.

You will need the following modules to solve the tasks:

In [1]:
import math
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from itertools import product

In [19]:
penguin_dataset = pd.read_csv("data/penguins_filtered.csv")

penguin_dataset = penguin_dataset.replace({
    "Adelie": 1,
    "Chinstrap" : 2,
    "Gentoo": 3,
    "male" : 0,
    "female" : 1,
    "Biscoe" : 0,
    "Dream" : 1,
    "Torgersen" : 2})

discrete_penguin_dataset = penguin_dataset[["sex", "island", "species"]]
print(penguin_dataset)
print(discrete_penguin_dataset.head())

     sex  island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0      0       2            39.1           18.7              181.0   
1      1       2            39.5           17.4              186.0   
2      1       2            40.3           18.0              195.0   
3      1       2            36.7           19.3              193.0   
4      0       2            39.3           20.6              190.0   
..   ...     ...             ...            ...                ...   
328    0       1            55.8           19.8              207.0   
329    1       1            43.5           18.1              202.0   
330    0       1            49.6           18.2              193.0   
331    0       1            50.8           19.0              210.0   
332    1       1            50.2           18.7              198.0   

     body_mass_g  species  
0         3750.0        1  
1         3800.0        1  
2         3250.0        1  
3         3450.0        1  
4         3650.0   

## I. Naive and Joint Bayes (3.5 points; 0.15 bonus per week)

1. Calculate the prior probabilities for the target feature. Transform them by applying the natural logarithm. 

In [7]:
# solution here
species, species_nr = np.unique(penguin_dataset["species"], return_counts=True)
p_species = []
for specie_nr in species_nr:
    p_species.append(specie_nr / sum(species_nr))

print(p_species)

for i in range(len(p_species)):
    p_species[i] = math.log(p_species[i], 10)

print(p_species)

[0.43843843843843844, 0.2042042042042042, 0.35735735735735735]
[-0.3580913777218827, -0.6899353208000835, -0.4468972721137891]


2. Write the formulas used to calculate the maximum aposteriori probability using Naive Bayes and Joint Bayes. Use the names of the variables from the discrete penguin dataset.

*Answer here*:

In [ ]:
#NAIVE BAYES: v_MAP = argmax P(a_i|v_j)P(v_j), where a_i is an attribute and v_j is target atrribute value
#JOINT BAYES: v_MAP = argmax P(a_1,a_2,...,a_n|v_j)P(v_j)

3. Find and calculate the logarithm of all the conditional probabilities (also names likelihoods) used to predict the label for the instance `{"sex" : 1, "island" : 2}` in Naive Bayes.

In [23]:
# solution here
#P(sex=1|species=1),P(island=2|species=1)
#P(sex=1|species=2),P(island=2|species=2)
#P(sex=1|species=3),P(island=2|species=3)

def calculate_conditional_probability(dataset, feature, target_feature, alpha):
    feature_vals, nr_feature_vals = np.unique(dataset[feature], return_counts=True)
    tf_name = list(target_feature.keys())[0]
    tf_val = target_feature[tf_name]
    for feature_val in feature_vals:
        temp_dataset = dataset[dataset[feature] == feature_val]
        target_feature_vals, nr_target_feature_vals = np.unique(temp_dataset[tf_name], return_counts=True)
        i_tf_val = 0
        #print(target_feature_vals, nr_target_feature_vals)
        for i in range(len(target_feature_vals)):
            if target_feature_vals[i] == tf_val:
                i_tf_val = i
                break
        if alpha==0:
            p_tf_val = nr_target_feature_vals[i_tf_val] / sum(nr_target_feature_vals)
        else:
            p_tf_val = (nr_target_feature_vals[i_tf_val] + alpha) / (sum(nr_target_feature_vals) + len(target_feature_vals))
        res_string = "P(" + tf_name + "=" + str(tf_val) + "|" + feature + "=" + str(feature_val) + ") = " + str(p_tf_val)
        print(res_string)
        print("ln(" + res_string + ") = " + str(math.log(p_tf_val, 10)))
        print()


calculate_conditional_probability(penguin_dataset,"species",{ "sex" : 1},0)
calculate_conditional_probability(penguin_dataset,"species",{ "island" : 2},0)

P(sex=1|species=1) = 0.5
ln(P(sex=1|species=1) = 0.5) = -0.30102999566398114

P(sex=1|species=2) = 0.5
ln(P(sex=1|species=2) = 0.5) = -0.30102999566398114

P(sex=1|species=3) = 0.48739495798319327
ln(P(sex=1|species=3) = 0.48739495798319327) = -0.3121189678295935

P(island=2|species=1) = 0.3219178082191781
ln(P(island=2|species=1) = 0.3219178082191781) = -0.4922549978487196

P(island=2|species=2) = 1.0
ln(P(island=2|species=2) = 1.0) = 0.0

P(island=2|species=3) = 1.0
ln(P(island=2|species=3) = 1.0) = 0.0



4. Why does the results contain infinity? Fix the calculation by using the Laplace Smoothing with `alpha = 1`.

In [24]:
# solution here
calculate_conditional_probability(penguin_dataset,"species",{ "sex" : 1},1)
calculate_conditional_probability(penguin_dataset,"species",{ "island" : 2},1)

P(sex=1|species=1) = 0.5
ln(P(sex=1|species=1) = 0.5) = -0.30102999566398114

P(sex=1|species=2) = 0.5
ln(P(sex=1|species=2) = 0.5) = -0.30102999566398114

P(sex=1|species=3) = 0.48760330578512395
ln(P(sex=1|species=3) = 0.48760330578512395) = -0.3119333586743059

P(island=2|species=1) = 0.3221476510067114
ln(P(island=2|species=1) = 0.3221476510067114) = -0.4919450310366868

P(island=2|species=2) = 1.0
ln(P(island=2|species=2) = 1.0) = 0.0

P(island=2|species=3) = 1.0
ln(P(island=2|species=3) = 1.0) = 0.0



5. Calculate the aposteriori probabilities of the labels and decide which label will Naive Bayes predict for the instance. Use only the logarithm values.

In [6]:
# solution here
#P(sex=1|species=1)P(island=2|species=1)P(species=1)
#P(sex=1|species=2)P(island=2|species=2)P(species=2)
#P(sex=1|species=3)P(island=2|species=3)P(species=3)





6. **Naive Bayes implemenation**: write a function called `naive_bayes` that takes three arguments:
- `df`: the dataframe which will be used for training
- `index_target`: the index of the column associated with the target feature
- `alpha`: the parameter used for Laplace Smoothing

The function should return a dictionary with the following fields:
- `log_prior`: the logarithmic values of the prior probabilities (the probability of the labels)
- `log_likelihoods`: a n x m x t array, where n - the number of features; m - the number of labels; t - the number of values for a feature; this array will contain the logarithmic values of the likelihoods (P(feature = value | target_feature = label))
- `n_classes`: the number of labels
- `n_feature_classes`: a vector that contains the number of unique values for each attribute
- `classes`: the name of the labels (the values of the target feature).

In [7]:
def naive_bayes(df, index_target = -1, alpha = 1e-10):
    pass 

7. Train the discrete penguin dataset using your version of Naive Bayes and sklearn's. Compare the values of the parameters.(Be careful at what type of Naive Bayes classifier you pick from sklearn!)

In [8]:
# solution here

8. Create a function named `nb_predict_prob` that uses the log probabilities calculated by Naive Bayes to infer the aposteriori probability of a new instance `X`.

In [9]:
def nb_predict_prob(nb_dict, X, use_log = False):
    pass

9. Create a function that does the Naive Bayes prediction using the Maximum Aposteriori Probability.

In [10]:
def nb_predict(nb_dict, X):
    pass

10. Create a function that calculate the accuracy of the trained model on a set of instances `X` with known labels `y`.

In [11]:
def nb_score(nb_dict, X, y):
    pass

11. Calculate the training accuracy of your Naive Bayes algorithm. Explain the results.

In [12]:
# solution here

12. Find and calculate all the conditional probabilities (also names likelihoods) used to predict the label for the instance `{"sex" : 1, "island" : 2}` in Joint Bayes. (*Hint*: panda's query function might provide itself useful.)

In [13]:
# solution here

13. Calculate the aposteriori probabilities of the labels and decide which label will Joint Bayes predict for the instance. Use the conditional and prior probabilities (*not* the logarithmic values).

In [14]:
# solution here

14. **Joint Bayes implemenation**: write a function called `joint_bayes` that takes two arguments:
- `df`: the dataframe which will be used for training
- `index_target`: the index of the column associated with the target feature

The function should return a dictionary with the following fields:
- `prior_probs`: the prior probabilities (the probability of the labels)
- `likelihoods`: a n x m array, where n - the number of labels; m - the number of combination between the values of the features; each label will have assigned a list containing the joint probability P(feature_1 = value_1, feature_2 = value_2, ...,  feature_n = value_n | target_feature = label)
- `n_classes`: the number of labels
- `n_feature_classes`: a vector that contains the number of unique values for each attribute
- `classes`: the name of the labels (the values of the target feature).

*Hint*: check the imports from the first cell of this notebook.

In [15]:
def joint_bayes(df, index_target = -1):
    pass

15. Train the Joint Bayes algorithm on the discrete penguin dataset. Print the obtained dictionary.

In [16]:
# solution here

16. Similarly to Naive Bayes, write the functions used to predict the aposteriori probabilities, the label and the accuracy of the Joint Bayes algorithm.

In [17]:
def jb_predict_prob(jb_dict, X):
    pass

def jb_predict(jb_dict, X):
    pass

def jb_score(jb_dict, X, y):
    pass


17. Calculate the training accuracy of your Joint Bayes algorithm. 

In [18]:
# solution here

## II. kNN (2 points; 0.15 bonus per week)

For this section we will use the entire `penguin_dataset`.

1. Calculate the Euclidean distance between the test instance `{"sex" : 1, "island" : 2, "bill_length" : 20, "bill_depth" : 40, "flipper_length" : 355, "body_mass" : 855}` and the instances from the dataset. Store the values in an object called `instance_distance`. Print the average distance. (*Hint*: check the norm function from the numpy package.)

In [19]:
# solution here

2. Find the `5` nearest neighbours of the test instance.

In [20]:
# solution here

3. Determine the probabilities of the labels that kNN would assign for this test instance (`k = 5`). Which label has the highest probability?

In [21]:
# solution here

4. Suppose that kNN gives for each neighbour a weight that is equal to the inverse of the distance. Print the changed probabilities, as well as the label predicted by kNN.

In [22]:
# solution here

5. **k-NN implementation**: Create a function `knn_predict_prob` that will take six arguments:
- `df`: the dataframe containing the features and the target feature
- `test_x`: a list with the attributes observed for *one* instance
- `k`: the number of nearest neighbours
- `use_weights`: boolean value that indicates whether to assign weights based on the inverse of the distance or not
- `p`: either an integer, indicating the order of the Minkowski distance (p=2 is the equivalent for Euclidean) or a custom distance function
- `index_target`: the index of the column that contains the labels of the target feature

The function should:
- calculate the distance between `test_x` and the observations from the dataset
- extract the k-nearest neighbours
- calculate the weight for each label
- normalize the weights to become probabilities
- return the probability vector, that will indicate the probability of the instance `test_x` to have the label `i`.

In [23]:
def knn_predict_prob(df, test_x, k, use_weights = True, p = 2, index_target = -1):
   pass

6. Write a function that, based on the probabilities calculated above, returns the label with the highest probability.

In [24]:
def knn_predict(df, test_x, k, use_weights = True, p = 2, index_target = -1):
    pass

7. Calculate the probabilities and the predicted label for the instance from exercise 1 using the following configurations:
- `k = 11, unweighted, Euclidean distance`
- `k = 11, weighted, Euclidean distance`
- `k = 11, unweighted, Manhattan distance`
- `k = 11, weighted, Manhattan distance`

Compare your results with the results obtained by the `sklearn` implementation.

In [25]:
# solution here

8. Write a function that calculates the accuracy of the kNN algorithm.

In [26]:
def knn_score(df, test_x, test_y, k, use_weights = True, p = 2, index_target = -1):
    pass

9. What is the training accuracy of the unweighted kNN when k varies from 3 to 15? (use only odd numbers). Does adding the weight / changing the distance metric improve the score? Justify.

In [27]:
# solution here

## III. AdaBoost (3 points; 0.25 bonus per week)

## IV. Testing the performance (1.5 points; 0.1 bonus per week)

# Exercise grading

| Section | Exercise | Points |
| --- | --- | --- |
| I |	1 | 	0.1 |
| I |	2 | 	0.1 |
| I |	3 | 	0.25 |
| I |	4 | 	0.25 |
| I |	5 | 	0.15 |
| I |	6 | 	0.65 |
| I |	7 | 	0.1 |
| I |	8 | 	0.25 |
| I |	9 | 	0.15 |
| I |	10 | 	0.1 |
| I |	11 | 	0.1 |
| I |	12 | 	0.1 |
| I |	13 | 	0.15 |
| I |	14 | 	0.65 |
| I |	15 | 	0.1 |
| I |	16 | 	0.2 |
| I |	17 | 	0.1 |
|II | 	1 | 	0.2 |
|II | 	2 | 	0.1 |
|II | 	3 | 	0.15 |
|II | 	4 | 	0.2 |
|II | 	5 | 	0.72 |
|II | 	6 | 	0.1 |
|II | 	7 | 	0.28 |
|II | 	8 | 	0.1 |
|II | 	9 | 	0.15 |
